In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [2]:
# 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows의 경우 '맑은 고딕'
# Mac의 경우: plt.rcParams['font.family'] = 'AppleGothic'
# Linux의 경우 적절한 한글 폰트를 설치해야 함 (예: 'NanumGothic')

# 음수 기호가 깨지는 문제 해결 (minus 폰트 설정)
plt.rcParams['axes.unicode_minus'] = False

In [3]:
# 두 개의 Excel 파일을 읽습니다.
df1 = pd.read_excel('pre_input_316LDSB.xlsx')
df2 = pd.read_excel('pre_defect_316LDSB.xlsx')

In [4]:
# Convert string column to year-month format
def convert_to_year_month(value):
    if isinstance(value, int):
        year = str(value)[:4]
        month = str(value)[4:]
        return f'{year}-{month.zfill(2)}'
    elif isinstance(value, str):
        year = value[:4]
        month = value[4:]
        return f'{year}-{month.zfill(2)}'
    else:
        return value

df1['년월'] = df1['년월'].apply(convert_to_year_month)

def convert_to_Rolling_year_month_date(date):
    year = date.year
    month = date.month
    return f'{year}-{month:02}' 


df1['압연년월'] = df1['최종압연일자'].apply(convert_to_Rolling_year_month_date)

# 압연년월을 datetime으로 변환 후 연도-월만 추출
df1['압연년월'] = pd.to_datetime(df1['압연년월']).dt.strftime('%Y-%m')

# 압연년월+heat_no
df1['압연년월_Heat'] = df1['압연년월']+'_'+df1['HEAT_NO'].astype(str)

df1.head()

,압연년,압연월,년월,HEAT_NO,LOT_NO,공정코드,구분,공장명,주조구분,재료코드,...,주여구분,지시GR종류,생산GR종류,생산GR공정,제강검사완료일,작업일,강관소재여부,주설비구분,압연년월,압연년월_Heat
0,2023,8,2023-08,S54522,B360555301,9Y01,압연,산세,T,7730.0,...,1,A,A,8301,2023-06-01,2023-08-31,N,C,2023-08,2023-08_S54522
1,2023,8,2023-08,S54522,B360555300,9Y01,압연,산세,T,7730.0,...,1,A,A,8301,2023-06-01,2023-08-31,N,C,2023-08,2023-08_S54522
2,2023,8,2023-08,S54522,B360555204,9Y01,압연,산세,T,7730.0,...,1,A,A,8301,2023-06-01,2023-08-31,N,C,2023-08,2023-08_S54522
3,2023,8,2023-08,S54522,B360555203,9Y01,압연,산세,T,7730.0,...,1,A,A,8301,2023-06-01,2023-08-31,N,C,2023-08,2023-08_S54522
4,2023,8,2023-08,S54522,B360555202,9Y01,압연,산세,T,7730.0,...,1,A,A,8301,2023-06-01,2023-08-31,N,C,2023-08,2023-08_S54522


In [5]:
def convert_to_year_month_date(date):
    year, month, _ = date.split('-')
    return f'{year}-{month}'

def convert_to_Rolling_year_month_date(date):
    year = date.year
    month = date.month
    return f'{year}-{month:02}' 

df2['년월'] = df2['년월일'].apply(convert_to_year_month_date)
df2['압연년월'] = df2['최종압연일자'].apply(convert_to_Rolling_year_month_date)

# 압연년월을 datetime으로 변환 후 연도-월만 추출
df2['압연년월'] = pd.to_datetime(df2['압연년월']).dt.strftime('%Y-%m')

# 압연년월+heat_no
df2['압연년월_Heat'] = df2['압연년월']+'_'+df2['HEAT_NO'].astype(str)

# 년월은 년월일 컬럼 앞에 배치
position = df2.columns.get_loc('년월일')
df2.insert(position, '년월', df2.pop('년월'))

df2.head()

,압연년,압연월,년월,년월일,업무구분,공장구분,HEAT_NO,LOT_NO,PON,공정순위,...,압연폭4,최종압연공정,최종압연일자,제품압연작업조,생성일시,HCR,투입중량,주설비구분,압연년월,압연년월_Heat
0,2023,9,2023-09,2023-09-22,2,9,S54805,B360593201,D23W9W1414,80,...,0,8100,2023-09-21,B,2023-10-05,NaN,NaN,C,2023-09,2023-09_S54805
1,2023,9,2023-09,2023-09-22,2,9,S54805,B360593201,D23W9W1414,80,...,0,8100,2023-09-21,B,2023-10-05,NaN,NaN,C,2023-09,2023-09_S54805
2,2023,7,2023-07,2023-07-24,2,9,S54204,B350698202,D23W7W1114,90,...,0,8100,2023-07-23,B,2023-08-07,NaN,NaN,C,2023-07,2023-07_S54204
3,2024,4,2024-04,2024-04-29,2,9,S57283,B440412302,D24W3W1469,80,...,0,8100,2024-04-28,C,2024-05-03,NaN,NaN,C,2024-04,2024-04_S57283
4,2024,4,2024-04,2024-04-29,2,9,S57283,B440412201,D24W4W1343,80,...,0,8100,2024-04-28,C,2024-05-03,NaN,NaN,C,2024-04,2024-04_S57283


In [6]:
# 데이터 병합
df = pd.concat([df1, df2], ignore_index=True)

# 병합된 데이터를 새로운 파일로 저장
df.to_excel('df_b.xlsx', index=False)